<a href="https://colab.research.google.com/github/Ppkojcha/swu-ds525/blob/main/etl_local0032.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETL with Spark (Local)

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
data = "github_events_01.json"

In [ ]:
spark = SparkSession.builder \
    .appName("ETL") \
    .getOrCreate()

In [ ]:
data_folder = "data"

In [ ]:
data = spark.read.option("multiline", "true").json(data_folder)

In [ ]:
data.show(10)

+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+--------------------+
|               actor|          created_at|         id|                 org|             payload|public|                repo|                type|
+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+--------------------+
|{https://avatars....|2022-08-17T15:52:40Z|23487963576|{https://avatars....|{started, null, n...|  true|{6296790, spring-...|          WatchEvent|
|{https://avatars....|2022-08-17T15:52:40Z|23487963624|                null|{null, null, null...|  true|{525860969, gurra...|         CreateEvent|
|{https://avatars....|2022-08-17T15:52:40Z|23487963529|                null|{null, e80c84c7bb...|  true|{350706029, afbel...|           PushEvent|
|{https://avatars....|2022-08-17T15:52:40Z|23487963558|{https://avatars....|{created, null, {...|  true|{226399669, CM

In [ ]:
data.printSchema()

root
 |-- actor: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- display_login: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- org: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- payload: struct (nullable = true)
 |    |-- action: string (nullable = true)
 |    |-- before: string (nullable = true)
 |    |-- comment: struct (nullable = true)
 |    |    |-- author_association: string (nullable = true)
 |    |    |-- body: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- html_url: string (nullable 

In [ ]:
data.select("id", "type","created_at").show(10)

+-----------+-----------------+
|         id|             type|
+-----------+-----------------+
|23487929637|IssueCommentEvent|
+-----------+-----------------+



In [ ]:
data.createOrReplaceTempView("staging_events")

In [ ]:
table = spark.sql("""
    select
        *
        
    from
        staging_events
""").show(10)

+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+-----------------+
|               actor|          created_at|         id|                 org|             payload|public|                repo|             type|
+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+-----------------+
|{https://avatars....|2022-08-17T15:51:05Z|23487929637|{https://avatars....|{created, {COLLAB...|  true|{75340147, 350org...|IssueCommentEvent|
+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+-----------------+



In [ ]:
table = spark.sql("""
    select
        id
        , type
        , created_at
        , to_date(created_at) as date
        , year(created_at) as year
        , actor.login
        , actor.url as actor_url
        , repo.name
        , repo.url as repo_url
        
    from
        staging_events
""")

In [ ]:
table.show(10)

+-----------+-----------------+--------------------+----------+----+-------+--------------------+-----------------+--------------------+
|         id|             type|          created_at|      date|year|  login|           actor_url|             name|            repo_url|
+-----------+-----------------+--------------------+----------+----+-------+--------------------+-----------------+--------------------+
|23487929637|IssueCommentEvent|2022-08-17T15:51:05Z|2022-08-17|2022|sukhada|https://api.githu...|350org/ak_intl_v3|https://api.githu...|
+-----------+-----------------+--------------------+----------+----+-------+--------------------+-----------------+--------------------+



In [ ]:
table = spark.sql("""
    select
        id
        , type
        , created_at
        , day(created_at) as day
        , month(created_at) as month
        , year(created_at) as year
        , date(created_at) as date
    from
        staging_events
""")

In [ ]:
table.show(10)

+-----------+-----------------+--------------------+---+-----+----+----------+
|         id|             type|          created_at|day|month|year|      date|
+-----------+-----------------+--------------------+---+-----+----+----------+
|23487929637|IssueCommentEvent|2022-08-17T15:51:05Z| 17|    8|2022|2022-08-17|
+-----------+-----------------+--------------------+---+-----+----+----------+



In [ ]:
table_events = spark.sql("""
    select
        id
        , type
        , created_at
        , day(created_at) as day
        , month(created_at) as month
        , year(created_at) as year
        , date(created_at) as date
    from
        staging_events
""")


In [ ]:
table_events.show(10)

In [ ]:
table_actors = spark.sql("""
    select
        actor.id as actor_id
        , actor.display_login as display_name
        
    from
        staging_events
""")



In [ ]:
table_actors.show(10)

In [ ]:
table_repo = spark.sql("""
    select
        repo.id as repo_id
        , repo.name
       
    from
        staging_events
""")


In [ ]:
table_repo.show(5)